In [2]:
#Sentimental Analysis - using fine tuning
import pandas as pd

books=pd.read_csv("books_with_categories.csv")

In [3]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k=None,
                      device=-1,
                      )
classifier("I love this!")

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528691716492176},
  {'label': 'neutral', 'score': 0.005764589179307222},
  {'label': 'anger', 'score': 0.004419791977852583},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.0004138525982853025}]]

In [11]:
books["description"][0]

"Emma Watson a research physician has been training for the mission of a lifetime: to study living organisms in the microgravity of space. But the true and lethal nature of the experiment has not been revealed to NASA and once aboard the space station things start to go wrong. A culture of single-celled Archaeons, gathered from the deep sea, begin to rapidly multiply and infect the crew - with deadly and agonising results. As her estranged husband and ground crew at NASA work against the clock to launch a rescue Emma stuggles to contain the lethal microbe. But with the contagion threatening Earth's population, there are those who would leave the astronauts stranded in orbit, quarantined aboard the station."

In [12]:
classifier(books["description"][0]) #classifing first one

[[{'label': 'disgust', 'score': 0.44187644124031067},
  {'label': 'fear', 'score': 0.2832786440849304},
  {'label': 'neutral', 'score': 0.16197003424167633},
  {'label': 'sadness', 'score': 0.05621432512998581},
  {'label': 'anger', 'score': 0.0484253466129303},
  {'label': 'surprise', 'score': 0.005015256814658642},
  {'label': 'joy', 'score': 0.003219984471797943}]]

In [5]:
#mixing emotion so- splitting description to individual  sentence and classfiy
sentences=books["description"][0].split(".")
predictions= classifier(sentences)

In [16]:
sentences[0]

'Emma Watson a research physician has been training for the mission of a lifetime: to study living organisms in the microgravity of space'

In [17]:
predictions[0]

[{'label': 'neutral', 'score': 0.6422688364982605},
 {'label': 'disgust', 'score': 0.12185034900903702},
 {'label': 'fear', 'score': 0.09685871005058289},
 {'label': 'surprise', 'score': 0.06819461286067963},
 {'label': 'joy', 'score': 0.035124193876981735},
 {'label': 'sadness', 'score': 0.019500814378261566},
 {'label': 'anger', 'score': 0.016202468425035477}]

In [18]:
sentences[3]

' As her estranged husband and ground crew at NASA work against the clock to launch a rescue Emma stuggles to contain the lethal microbe'

In [19]:
predictions[3]

[{'label': 'fear', 'score': 0.4600585401058197},
 {'label': 'neutral', 'score': 0.19147056341171265},
 {'label': 'sadness', 'score': 0.1358918994665146},
 {'label': 'anger', 'score': 0.12885285913944244},
 {'label': 'disgust', 'score': 0.07337749004364014},
 {'label': 'surprise', 'score': 0.006486775819212198},
 {'label': 'joy', 'score': 0.003861889475956559}]

In [21]:
predictions

[[{'label': 'neutral', 'score': 0.6422688364982605},
  {'label': 'disgust', 'score': 0.12185034900903702},
  {'label': 'fear', 'score': 0.09685871005058289},
  {'label': 'surprise', 'score': 0.06819461286067963},
  {'label': 'joy', 'score': 0.035124193876981735},
  {'label': 'sadness', 'score': 0.019500814378261566},
  {'label': 'anger', 'score': 0.016202468425035477}],
 [{'label': 'anger', 'score': 0.3963668942451477},
  {'label': 'fear', 'score': 0.38848766684532166},
  {'label': 'disgust', 'score': 0.10238470137119293},
  {'label': 'sadness', 'score': 0.058679379522800446},
  {'label': 'neutral', 'score': 0.0485905222594738},
  {'label': 'surprise', 'score': 0.0035759860184043646},
  {'label': 'joy', 'score': 0.0019147975835949183}],
 [{'label': 'fear', 'score': 0.448210209608078},
  {'label': 'disgust', 'score': 0.36927053332328796},
  {'label': 'neutral', 'score': 0.08002244681119919},
  {'label': 'anger', 'score': 0.07518163323402405},
  {'label': 'sadness', 'score': 0.0154481269

In [6]:
# for each book separate column for each of emotions classes - high prob across description- sorting first
sorted(predictions[0],key=lambda x:x["label"])

[{'label': 'anger', 'score': 0.009156367741525173},
 {'label': 'disgust', 'score': 0.0026284768246114254},
 {'label': 'fear', 'score': 0.06816227734088898},
 {'label': 'joy', 'score': 0.04794260486960411},
 {'label': 'neutral', 'score': 0.14038598537445068},
 {'label': 'sadness', 'score': 0.002122163772583008},
 {'label': 'surprise', 'score': 0.7296021580696106}]

In [8]:
import numpy as np

emotion_labels = ["anger","disgust","fear","happiness","sadness","surprise"] #key->col of pandas df

isbn = []

emotion_score = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction_per_sentence in predictions:#looping sentence
        sorted_predictions = sorted(prediction_per_sentence, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):#extract score in each sentence for each emotion
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}


In [9]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_score = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_score[label].append(max_score[label])

In [51]:
emotion_score

{'anger': [np.float64(0.3963668942451477),
  np.float64(0.4385429918766022),
  np.float64(0.07747067511081696),
  np.float64(0.2532133460044861),
  np.float64(0.06413363665342331),
  np.float64(0.0077097732573747635),
  np.float64(0.06413363665342331),
  np.float64(0.10218361020088196),
  np.float64(0.06413363665342331),
  np.float64(0.0287205558270216)],
 'disgust': [np.float64(0.36927053332328796),
  np.float64(0.6104852557182312),
  np.float64(0.10400673747062683),
  np.float64(0.16550090909004211),
  np.float64(0.10400673747062683),
  np.float64(0.011465298943221569),
  np.float64(0.23246851563453674),
  np.float64(0.12519362568855286),
  np.float64(0.10400673747062683),
  np.float64(0.009238985367119312)],
 'fear': [np.float64(0.4600585401058197),
  np.float64(0.8927624821662903),
  np.float64(0.824425458908081),
  np.float64(0.9207272529602051),
  np.float64(0.051362838596105576),
  np.float64(0.008247917518019676),
  np.float64(0.13417752087116241),
  np.float64(0.70274233818054

In [10]:
from tqdm import tqdm

emotion_labels = ["anger","disgust","fear","happiness","sadness","surprise"]
isbn = []
emotion_score = {label: [] for label in emotion_labels}


for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_score = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_score[label].append(max_score[label])

100%|██████████| 5197/5197 [15:21<00:00,  5.64it/s]


In [13]:
emotions_df=pd.DataFrame(emotion_score)
emotions_df["isbn13"] = isbn

In [14]:
emotions_df

,anger,disgust,fear,happiness,sadness,surprise,isbn13
0,0.064134,0.273591,0.928168,0.932798,0.646216,0.967158,9780002005883
1,0.612619,0.348284,0.942528,0.704422,0.887940,0.111690,9780002261982
2,0.064134,0.104007,0.972321,0.767239,0.549477,0.111690,9780006178736
3,0.351485,0.150722,0.360705,0.251882,0.732685,0.111690,9780006280897
4,0.081412,0.184496,0.095043,0.040564,0.884390,0.475880,9780006280934
...,...,...,...,...,...,...,...
5192,0.148209,0.030643,0.919165,0.255171,0.853721,0.980877,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,9788173031014
5194,0.009997,0.009929,0.339218,0.947779,0.375754,0.066685,9788179921623
5195,0.064134,0.104007,0.459270,0.759457,0.951104,0.368111,9788185300535


In [15]:
#merging to book df
books=pd.merge(books,emotions_df,on="isbn13")

In [16]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,happiness,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928168,0.932798,0.646216,0.967158
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web:A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348284,0.942528,0.704422,0.887940,0.111690
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767239,0.549477,0.111690
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351485,0.150722,0.360705,0.251882,0.732685,0.111690
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184496,0.095043,0.040564,0.884390,0.475880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148209,0.030643,0.919165,0.255171,0.853721,0.980877
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375754,0.066685
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that:Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459270,0.759457,0.951104,0.368111


In [17]:
#saving df
books.to_csv("books_with_emotions.csv", index=False)